In [74]:
import pandas as pd
from sklearn.metrics import r2_score
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [75]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("test_label/y_test.csv")

In [76]:
X_test.head(3)

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,12000,merc,GLS Class,2017,Automatic,12046,Diesel,150.0,37.2,3.0
1,12001,vw,Amarok,2017,Automatic,37683,Diesel,260.0,36.2,3.0
2,12004,merc,GLS Class,2019,Automatic,10000,Diesel,145.0,34.0,3.0


In [4]:
with open('model_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [5]:
predictions = pipeline.predict(X_test)
#pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)

In [6]:
predictions

array([40909.17, 23970.39, 63569.32, ..., 20766.01, 20385.71,  2874.1 ])

In [7]:
r2_score(y_test['price'], predictions)

0.9503618905894328

In [34]:
import urllib.request
import json      

def get_prediction(brand, model, year, transmission, mileage, fuelType, tax, mpg, engineSize):
    body = {"brand": brand,
            "model": model,
            "year": year,
            "transmission": transmission,
            "mileage": mileage,
            "fuelType": fuelType,
            "tax": tax,
            "mpg": mpg,
            "engineSize": engineSize} 

    myurl = "http://192.168.0.106:8180/predict"
    req = urllib.request.Request(myurl)
    
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [41]:
get_prediction('merc', 'GLS Class',2017,'Automatic',12046,'Diesel',150.0,37.2,3.0)

38834.58

In [68]:
%%time
predictions = X_test.loc[:, X_test.columns != 'carID'].apply(lambda x: get_prediction(*x), 1)

Wall time: 39 s


In [71]:
r2_score(y_test['price'], predictions)

0.9503618905894328